In [27]:
import librosa
import numpy as np
import os
import json
import pickle

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from micromlgen import port

In [7]:
DATASET_PATH = "mini_speech_commands"
SAMPLE_RATE = 22050
JSON_PATH = "data.json"

# preprocess data
def preprocess_dataset(dataset_path, json_path, n_mfcc=13, n_fft=2048, hop_length=512):
  
      # dictionary to store data
      data = {
          "mapping": [],
          "labels": [],
          "mfcc": [],
          "files": []
      }
  
      # loop through all sub-dirs
      for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
  
          # ensure we're not at root level
          if dirpath is not dataset_path:
  
              # save label (i.e., sub-dir name) in the mapping
              dirpath_components = dirpath.split("/") # "mini_speech_commands/down" => ["mini_speech_commands", "down"]
              semantic_label = dirpath_components[-1]
              data["mapping"].append(semantic_label)
              print("\nProcessing: '{}'".format(semantic_label))
  
              # process files for a specific sub-dir
              for f in filenames:
  
                  # load audio file
                  file_path = os.path.join(dirpath, f)
                  signal, sample_rate = librosa.load(file_path)

                  if len(signal) >= SAMPLE_RATE: # ensure consistency of the length of the signal
                    signal = signal[:SAMPLE_RATE]
  
                    # extract MFCCs
                    mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
                    mfcc = mfcc.T
  
                    # store data for analysed track
                    data["mfcc"].append(mfcc.tolist())
                    data["labels"].append(i-1)
                    data["files"].append(file_path)
                    # print("{}: {}".format(file_path, i-1))
  
      # save MFCCs to json file
      with open(json_path, "w") as fp:
          json.dump(data, fp, indent=4)
  
      print("Finished processing.")

if __name__ == "__main__":
  preprocess_dataset(DATASET_PATH, JSON_PATH)


Processing: 'mini_speech_commands\down'

Processing: 'mini_speech_commands\stop'

Processing: 'mini_speech_commands\up'
Finished processing.


In [8]:
# Load Data
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
  
    # convert lists into numpy arrays
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])
  
    return inputs, targets

In [32]:
# Train data using scikit learn
def train_test_model(dataset_path):
    # load data
    X, y = load_data(dataset_path)
  
    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # create network with linear regression
    model = LogisticRegression(
        solver='lbfgs',
        multi_class='multinomial',
        max_iter=100,
        fit_intercept=True,
        n_jobs=3,
        C=0.1,
        class_weight=None,
        intercept_scaling=1,
        penalty='l2',
        random_state=None,
        tol=0.0001,
        verbose=0,
        warm_start=False
    )
    # model = MLPClassifier(
    #     hidden_layer_sizes=(512, 256),
    #     activation="relu",
    #     solver="adam",
    #     batch_size=32,
    #     verbose=1,
    #     epsilon=1e-8,
    #     alpha=0.0001,
    #     learning_rate="adaptive",
    #     max_iter=100
    # )

    # reshape the 3d array to 2d array
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2])
    
    # Print some details
    print("X_train.shape: {}".format(X_train.shape))
    print("X_test.shape: {}".format(X_test.shape))
    print("y_train.shape: {}".format(y_train.shape))
    print("y_test.shape: {}".format(y_test.shape))

    # train network
    # model = LogisticRegression(max_iter=200)
    model.fit(X_train, y_train)
  
    # evaluate network
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

    # save the model using pickle
    pickle.dump(model, open("model.sav", 'wb'))

    #save the model using json format
    model_param = {}
    model_param["coef_"] = model.coef_.tolist()
    model_param["intercept_"] = model.intercept_.tolist()

    json_txt = json.dumps(model_param, indent=4)
    with open("model.json", "w") as json_file:
        json_file.write(json_txt)
    
    # export to plain C
    c_code = port(model)

    print("Accuracy: {:.2f}%".format(accuracy*100))
    
    print(c_code)
    

In [33]:
# Run the model
if __name__ == "__main__":
  print("Training model...")
  train_test_model(JSON_PATH)

Training model...
X_train.shape: (1877, 572)
X_test.shape: (805, 572)
y_train.shape: (1877,)
y_test.shape: (805,)
Accuracy: 72.92%
#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class LogisticRegression {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        float votes[3] = { -9.8508711e-05 ,2.3382433e-05 ,7.5126279e-05  };
                        votes[0] += dot(x,   0.002103151114  , -0.004174013742  , 0.000137079394  , 0.000280351992  , -0.001827274232  , 0.000525519693  , 0.000746516132  , 0.001144133242  , -0.000366074696  , -0.001194603308  , 0.00054621519  , 0.00072535641  , 0.001219795351  , -0.001106105708  , -0.002729587689  , 0.002307490871  , 0.001286329316  , -0.001421271799  , -0.001003483265  , 0.000372594145  , 0.000820680514  , -0.000486873493  , -0.001159249795  , 0.0